Draw the spatial pattern and violin plot of 500hpa as an example.

# Imports

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import proplot as pplt

# Data

## spatial pattern

In [11]:
eof_all_all = xr.open_dataset("/work/mh0033/m300883/3rdPanel/data/allPattern/ind/first_pattern/all_all_eof.nc")


In [13]:
eof_all_all = eof_all_all.eof

In [14]:
eof_500hpa = eof_all_all.sel(hlayers = 50000)

## index

In [2]:
all_all_index = xr.open_dataset('/work/mh0033/m300883/3rdPanel/data/allPattern/ind/all_pattern/all_all_nonstd.nc')

In [4]:
all_all_index = all_all_index.pc

standardize with the mean and std of whole

In [5]:
def standardize(index,dim = ('time','ens')):
    """
    standardardize with the mean and std of 'time' and 'ens'.
    """
    mean = index.mean(dim = dim)
    std = index.std(dim = dim)
    index = (index-mean)/std
    return index

In [6]:
all_all_index = standardize(all_all_index)

In [9]:
first10_all_whole = all_all_index.isel(time = slice(0,10))
last10_all_whole = all_all_index.isel(time = slice(-10,all_all_index.time.size))

In [18]:
index_500hpa_first = first10_all_whole.sel(hlayers = 50000)
index_500hpa_last = last10_all_whole.sel(hlayers = 50000)

# plot of spatial pattern and violin plot